# Comparison of an Interrupted Poisson Process with a M/M/1 system

## Imports

In [2]:
%matplotlib inline
import scipy.stats as sts
import numpy as np
from scipy.special import factorial
import matplotlib.pyplot as plt

from Simulation import Simulation

### Run the simulator with an interrupted poisson process

In [57]:
num_servers = 1
service_mean = 0.6
w1 = 10
w2 = 10
lmbda = 1

interarrival_exp = lambda num: sts.expon.rvs(size=num,scale=1./lmbda)
service_exp = lambda num: sts.expon.rvs(size=num,scale=service_mean)
stateGenOnToOff = lambda num: sts.expon.rvs(size=num,scale=w1)
stateGenOffToOn = lambda num: sts.expon.rvs(size=num,scale=w2)
stateGen = (stateGenOnToOff,stateGenOffToOn)

num_runs = 100
num_departs = 10000
sa_mean = np.zeros((num_runs,))
sa_var = np.zeros((num_runs,))
for run in range(num_runs):
    if (run+1) % 10 == 0:
        print("Run {} out of {}".format(run+1,num_runs))
    s = Simulation(num_servers=num_servers,interarrival=interarrival_exp,service=service_exp,
                   stateGen=stateGen,superpositions=1)
    while s.num_departed < num_departs:
        s.advanceTime()
    serviceActivityTimes = np.array(s.serviceActivityTimes)
    sa_mean[run] = serviceActivityTimes.mean()
    sa_var[run] = serviceActivityTimes.var(ddof=1)

Run 10 out of 100
Run 20 out of 100
Run 30 out of 100
Run 40 out of 100
Run 50 out of 100
Run 60 out of 100
Run 70 out of 100
Run 80 out of 100
Run 90 out of 100
Run 100 out of 100


### Results from above

In [56]:
sa_mean = sa_mean[~np.isnan(sa_mean)]
sa_var = sa_var[~np.isnan(sa_var)]

print("\nService activity mu mean: {0:.6f}".format(sa_mean.mean()))
print("Service activity mu var: {0:e}".format(sa_mean.var(ddof=1)))
n = sa_mean.shape[0]
inner = (sa_mean.std(ddof=1)/np.sqrt(n))
alpha = 0.05
CI = [sa_mean.mean()+inner*sts.t.ppf(alpha/2,df=n-1),sa_mean.mean()+inner*sts.t.ppf(1-(alpha/2),df=n-1)]
print("Service activity mu CI: [{0:.6f},{1:.6f}]".format(*CI))

print("\nService activity sigma2 mean: {0:.6f}".format(sa_var.mean()))
print("Service activity sigma2 var: {0:e}".format(sa_var.var(ddof=1)))
n = sa_var.shape[0]
inner = (sa_var.std(ddof=1)/np.sqrt(n))
alpha = 0.05
CI = [sa_var.mean()+inner*sts.t.ppf(alpha/2,df=n-1),sa_var.mean()+inner*sts.t.ppf(1-(alpha/2),df=n-1)]
print("Service activity sigma2 CI: [{0:.6f},{1:.6f}]".format(*CI))


Service activity mu mean: 4.361989
Service activity mu var: 1.037998e-02
Service activity mu CI: [4.341773,4.382204]

Service activity sigma2 mean: 36.691228
Service activity sigma2 var: 1.116754e+01
Service activity sigma2 CI: [36.028145,37.354311]


In [36]:
mu = (1/service_mean)
print("Theoretical mean Service activity: {0:.1f}".format(1/(mu-lmbda)))
print("Theoretical var Service activity: {0:.1f}".format((1+(lmbda/mu))/(mu**2*(1-(lmbda/mu))**3)))

Theoretical mean Service activity: 4.0
Theoretical var Service activity: 144.0
